## Gaussian Quadrature: Integration with orthogonal polynomials

See: https://en.wikipedia.org/wiki/Gauss%E2%80%93Legendre_quadrature

We approximate integrals as follows:

$$\int_a^b f(x)\ dx \approx \sum_{k=0}^{n} w_k f(x_k).$$

The nodes $x_k$ are the roots of the Legendre (or other orthogonal) polynomial.

I'll use weights from the Wikipedia table. Later I'll show you how to compute them yourself.


### 2-point Gaussian Quadrature

We want to weights $c_i$ and nodes $x_i$ that make this two point formula a good approximation to the integral

$$\int_{-1}^1 f(x)\ dx = c_1f(x_1) + c_2 f(x_2).$$

We anticipate that these 4 constants will allow us to compute the integral exactly for polynomials of degree 3 and lower. We write 4 equations in these four unknowns using the first 4 monomials.

$$\int_{-1}^1 1 \ dx = 2 = c_1 + c_2$$
$$\int_{-1}^1 x \ dx = 0 = c_1 x_1 + c_2 x_2$$
$$\int_{-1}^1 x^2 \ dx = \frac{2}{3} = c_1 x_1^2 + c_2 x_2^2$$
$$\int_{-1}^1 x^3 \ dx = 0 = c_1 x_1^3 + c_2 x_2^3$$

The solution is $c_1=c_2=1$ and $x_2 = -x_1 = \frac{1}{\sqrt{3}}$. The nodes are the roots of the first two Legendre polynomials and this relationship is true in general. (Proof in textbook.) This makes the system above linear.

In [34]:
using Polynomials
using SpecialPolynomials

In [6]:
roots(basis.(Legendre,2))

2-element Vector{ComplexF64}:
 -0.5773502691896257 + 0.0im
  0.5773502691896257 + 0.0im

For $n=2$ the weights are 1 and the integral is computed over the interval $[-1,1]$.

In [7]:
GL_quad_2(f) = f(-0.5773502691896257) + f(0.5773502691896257)

GL_quad_2 (generic function with 1 method)

In [63]:
using QuadGK

In [11]:
quadgk(cos, -1, 1)

(1.6829419696157932, 1.9984014443252818e-15)

In [12]:
GL_quad_2(cos)

1.6758236553899863

In [13]:
quadgk(exp, -1, 1)

(2.3504023872876028, 2.220446049250313e-15)

In [14]:
GL_quad_2(exp)

2.3426960879097307

In [16]:
GL_quad_2(x -> 1 + x + x^2)  # x + x^2/2 + x^3 / 3, integral is 8/3, this result is exact

2.6666666666666665

## Computing coefficients in Gauss-Legendre quadrature

One way to get the weights is to integrate the Lagrange interpolating polynomial (a polynomial with roots at the  Legendre nodes).

$$w_i = \int_{-1}^1 \frac{P_n(x)}{\prod_{k\neq i}x_i - x_k}.$$

In [56]:
LegendreCoefficient = function(i, n)
    # ci from Theorem 4.7
    r = real.(roots(basis.(Legendre, n)))
    xi = r[i]
    others = copy(r)  # Tricky -- I modify others below, so be sure it's different from roots
    deleteat!(others, i)
    p = fromroots(others) / prod(xi .- others)
    integrate(p, -1, 1)
end
LegendreWeights(n) = [ LegendreCoefficient(i, n) for i in 1:n ]  
@show LegendreWeights(2)
@show LegendreWeights(3)
@show LegendreWeights(4) # compare with Wikipedia table

LegendreWeights(2) = [1.0, 1.0]
LegendreWeights(3) = [0.5555555555555554, 0.8888888888888891, 0.5555555555555554]
LegendreWeights(4) = [0.3478548451374539, 0.6521451548625463, 0.652145154862546, 0.347854845137454]


4-element Vector{Float64}:
 0.3478548451374539
 0.6521451548625463
 0.652145154862546
 0.347854845137454

In [57]:
function G_quad(f,n) 
    r = real.(roots(basis.(Legendre, n)))
    w = LegendreWeights(n)
    w' * f.(r)
end

G_quad (generic function with 1 method)

In [58]:
@show G_quad(exp, 2)
@show G_quad(exp, 6)
@show G_quad(exp, 10);

G_quad(exp, 2) = 2.3426960879097307
G_quad(exp, 6) = 2.350402387286037
G_quad(exp, 10) = 2.350402387287624


To integrate over another interval, simply change variables.

In [59]:
gaussianQuadAB = function(f, a, b, n)
   G_quad( t -> f(((b-a)*t + b + a )/2), n) * (b-a)/2
end

#93 (generic function with 1 method)

In [65]:
gaussianQuadAB( x -> x^6 - x^2*sin(2*x), 1, 3, 10), quadgk( x -> x^6 - x^2*sin(2*x), 1, 3, order=10)

(317.344246673829, (317.3442466738263, 1.7053025658242404e-13))

In [92]:
gaussianQuadAB( x -> x^6 - x^2*sin(2*x), 1, 3, 117)

-2.536069089336828e41

### Textbook method

The textbook solves the linear system to find the weights and uses a convenient formula for the roots of the Legendre polynomials (nodes).

In [37]:
using LinearAlgebra
"""
    glint(f,n)

Perform Gauss-Legendre integration for the function `f` on `n` nodes
in (-1,1). Returns the integral estimate and a vector of the nodes used.
"""
function glint(f,n)
    # Nodes and weights are found via a tridiagonal eigenvalue problem.
    β = @. 0.5/sqrt(1-(2*(1:n-1))^(-2))
    T = diagm(-1=>β,1=>β)
    λ,V = eigen(T)
    p = sortperm(λ)
    x = λ[p]               # nodes
    c = @. 2V[1,p]^2       # weights

    # Evaluate the integrand and compute the integral.
    I = dot(c,f.(x))      # vector inner product
    return I,x
end

glint

In [38]:
glint(exp, 10)

(2.350402387287604, [-0.9739065285171705, -0.8650633666889841, -0.6794095682990241, -0.4333953941292462, -0.1488743389816315, 0.14887433898163216, 0.43339539412924755, 0.6794095682990245, 0.8650633666889843, 0.9739065285171716])

### Other benchmarks

Adaptive Simpson's rule (new implementation, less efficient, but easier to read).

In [66]:
function adaptive(f, a, b, maxN, tolerance)
    simpsonElement(f, a, b) = (b-a)/6*(f(a) + 4*f((a+b)/2) + f(b))
    Sab = simpsonElement(f, a, b)
    Sab2 = simpsonElement(f, a, (a+b)/2) + simpsonElement(f, (a+b)/2, b) 
    error = abs(Sab - Sab2)
    if error < 10*tolerance || maxN < 2
        Sab2
    else
        adaptive(f, a, (a+b)/2, maxN-1, tolerance) + adaptive(f, (a+b)/2, b, maxN-1, tolerance)
    end
end
adaptive( x -> x^6 - x^2*sin(2*x), 1, 3, 15, 1e-15)

317.3442466738266

The gauss function in QuadGK gives the nodes and weights we've been using.

In [69]:
gauss(2)

([-0.5773502691896258, 0.5773502691896258], [0.9999999999999998, 0.9999999999999998])

Here is a simple minded approach to adaptive Gaussian quadrature. The error control is very approximate.

In [72]:
function adaptiveQuad(f, a, b, n, maxN, tolerance)
    Sab = gaussianQuadAB(f, a, b, n)
    Sab2 = gaussianQuadAB(f, a, (a+b)/2, n) + gaussianQuadAB(f, (a+b)/2, b, n) 
    error = abs(Sab - Sab2)
    if error < tolerance || maxN < 2 
        Sab2 
    else
        adaptiveQuad(f, a, (a+b)/2, n, maxN-1, tolerance) + adaptiveQuad(f, (a+b)/2, b, n, maxN-1, tolerance)
    end
end

adaptiveQuad (generic function with 1 method)

In [73]:
adaptiveQuad( x -> x^6 - x^2*sin(2*x), 1, 3, 3, 15, 1e-15)

317.3442466738263

In [86]:
adaptiveQuad( x -> log(x), 1, 10_000, 3, 20, 1e-15)

82104.40371976182

In [83]:
quadgk(x -> log(x), 1, 10_000)

(82104.40371975201, 0.00011569353589990783)